In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 107.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 22.4 MB/s eta 0:00:00


In [2]:
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.3/800.3 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 KB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 54.4 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd

import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.core.lightning import LightningModule
from torch.utils.data import DataLoader, Dataset

from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

import re
import math
import random
from tqdm import tqdm

## (1) 데이터 전처리
문장부호 유지

In [4]:
df = pd.read_csv("/content/drive/MyDrive/데이터/인물 챗봇/택_최종.csv")

In [5]:
df.shape

(11849, 4)

In [6]:
df['qa'] = df['Q'] + df['A']

In [7]:
df.drop_duplicates(subset='qa', inplace = True, ignore_index=True )

In [8]:
df.head(5)

,Unnamed: 0,Q,A,role,qa
0,0,야 근데 너 언제왔냐?,학교 어제. 가?,택,야 근데 너 언제왔냐?학교 어제. 가?
1,1,야 근데 너 언제왔냐?,어제. 가? 학교,택,야 근데 너 언제왔냐?어제. 가? 학교
2,2,야 근데 너 언제왔냐?,어제. 학교 가?,택,야 근데 너 언제왔냐?어제. 학교 가?
3,6,언제 왔어?,학교 어제. 가?,택,언제 왔어?학교 어제. 가?
4,7,언제 왔어?,어제. 가? 학교,택,언제 왔어?어제. 가? 학교


In [9]:
df.drop(['Unnamed: 0', 'role', 'qa'], inplace = True, axis = 1)

In [10]:
df.shape

(9442, 2)

## (2) 모델링
GPU

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [12]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',pad_token='<pad>', mask_token='<mask>')
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [13]:
# 챗봇 데이터를 처리하는 클래스를 만든다.
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=40):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = "<usr>"
        self.a_token = "<sys>"
        self.bos = '</s>'
        self.eos = '</s>'
        self.pad = '<pad>'
        self.mask = '<mask>'
        self.tokenizer = tokenizer

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn["Q"]  # 질문 열을 가져온다.
        a = turn["A"]  # 답변 열을 가져온다.

        q_toked = self.tokenizer.tokenize(self.q_token + q  + self.bos)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        #질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len        # 최대길이 - 질문길이
            if a_len <= 0:       # 질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   # 질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        #질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.    
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        # 질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)

In [14]:
def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data).to(device), torch.LongTensor(mask).to(device), torch.LongTensor(label).to(device)

In [15]:
train_set = ChatbotDataset(df, max_len=40) 
train_dataloader = DataLoader(train_set, batch_size=32, num_workers=0, shuffle=True, collate_fn=collate_batch,)

In [16]:
model  = model.to(device)
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [17]:
learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epoch = 20
Sneg = -1e18

In [18]:
for epoch in tqdm(range(epoch)):
    for batch_idx, samples in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        out = model(token_ids)
        out = out.logits      #Returns a new tensor with the logit of the elements of input
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward()
        # 학습 끝
        optimizer.step()
print ("end")

  0%|          | 0/20 [00:00<?, ?it/s]<ipython-input-14-d9c5350f11d2>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  return torch.LongTensor(data).to(device), torch.LongTensor(mask).to(device), torch.LongTensor(label).to(device)
100%|██████████| 20/20 [35:30<00:00, 106.53s/it]

end


22분

In [19]:
torch.save(model, '/content/drive/MyDrive/데이터/인물 챗봇/택kogpt2.pt')

##(3) 챗봇 결과
NONE

In [ ]:
!pip install transformers

In [2]:
import torch

from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [4]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',pad_token='<pad>', mask_token='<mask>')
model = torch.load('/content/drive/MyDrive/데이터/인물 챗봇/택kogpt2.pt', map_location=device)

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [5]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(tokenizer.encode("<usr>" + q +"<sys>" + a)).unsqueeze(dim=0)
            input_ids.to(device)
            pred = model(input_ids)
            pred = pred.logits
            gen = tokenizer.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
            if gen == '</s>':
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(a.strip()))

user > 안녕
Chatbot > 안녕하세요.
user > 뭐해?
Chatbot > ...
user > 말을 해...
Chatbot > 말 쪽팔려서  안하고
user > 너 바둑 잘하지
Chatbot > 부탁하시니까.. 그래두
user > 바둑하는 방법 알려줘
Chatbot > 부탁하시니까.. 그래두
user > 배고프지
Chatbot > 고맙다. 그래.
user > 배고파
Chatbot > 먹었어요.. 지금 시간이 몇 신데요.
user > 나는 안 먹었어 밥
Chatbot > 이거 진주야, 너 먹어라.
user > 진주를 왜 먹어!
Chatbot > 어.. 나야.. 근데 정신이 없었어.. 미안해. 진짜 미안해.
user > 그래 미안
Chatbot > 미안해. 완전 미안하다.
user > 뭐가 미안해
Chatbot > 미안하다. 아아 까먹었다,
user > 엥 까먹었어?
Chatbot > 맞다. 희동이 내일 안 까먹었지?
user > 알지 희동이
Chatbot > 희동이 맞다.  너 내일 안 까먹었지?
user > 학교가야지
Chatbot > 야..우리 형 대학 가야돼...
user > 우리 내일 학교가
Chatbot > 내일 월,화 난 계속 대국 있는데.
user > 화이팅!
Chatbot > 아..!!
user > 잘하고와
Chatbot > 잘먹었어요..!!
user > 그래...
Chatbot > 아무튼 너 토요일날 시간 비워나, 대국 끝나고 전화 줄게
user > 뭐하려고?
Chatbot > 어.. 나 덕선아.
user > 뭐
Chatbot > 마니또.. 나  너야.
user > 그걸 말하면 어떡해
Chatbot > 어떡해요? 못하는데
user > 마니또는 몰래 하는 거야
Chatbot > 어?
user > 너 나 좋아하지
Chatbot > 나 좋아해.
user > 나도 너 좋아해
Chatbot > 나 좋아해.
user > 추워
Chatbot > 추워!! 아,
user > 나 춥다고
Chatbot > 추워. 싫어.
user > 나는 덕선이야
Chat

In [ ]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(tokenizer.encode("<usr>" + q +"<sys>" + a)).unsqueeze(dim=0)
            input_ids.to(device)
            pred = model(input_ids)
            pred = pred.logits
            gen = tokenizer.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
            if gen == '</s>':
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(a.strip()))

user > 안녕
Chatbot > 안녕하세요.
user > 택아 뭐해
Chatbot > 뭔 아빠!! 일 있으세요?
user > 나는 덕선이야
Chatbot > 덕선이야. ..
user > 바둑 잘해?
Chatbot > 영화. ..
user > 무슨 말이야...
Chatbot > 사랑. 영원한
user > 사랑해
Chatbot > 사랑해.
user > 학교가자
Chatbot > 저.. ..  덕선아. ..
user > 왜?
Chatbot > 우리. . 영화 볼까..?  영화 보자.
user > 무슨 영화?
Chatbot > 보고 영화 싶어.
user > 영화 보자
Chatbot > 오늘 중요한 못온대, 약속 있다고 내일온대, 요새 되는 일이 없다.
user > 배고프다
Chatbot > 먹었어.. 지금 시간이 몇 신데요.
user > 나는 안 먹었는데...
Chatbot > 안 네.  먹어요..
user > 좋아해
Chatbot > 좋아해. 응.... 나 덕선이
user > 추워
Chatbot > 코앞인데 아, 뭐. 괜찮아.
user > 나 추워
Chatbot > 아, 코앞인데 뭐. 괜찮아.
user > 나 안 괜찮은데
Chatbot > 안 되는데?
user > 졸려
Chatbot > 너 안돼.  운전해야지
user > 안전운전
Chatbot > 운전하고 내가 왔어.
user > quit
